In [5]:
import cv2
import PIL
import numpy as np
import os

from PIL import Image

In [2]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0):
            return img
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

In [3]:
def circle_crop(img, sigmaX):   
    """
    Create circular crop around image centre    
    """    
    img = crop_image_from_gray(img)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted(img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
    return img 

In [9]:
for grado in os.listdir('No Inpainting/No_Proc/iPhone'):
    for i in os.listdir('No Inpainting/No_Proc/iPhone/' + grado):
        img = cv2.imread('No Inpainting/No_Proc/iPhone/' + grado + '/' + i)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_t = circle_crop(img, 30)
        image = Image.fromarray(img_t, 'RGB')
        image.save('No Inpainting/Procesadas/iPhone/' + grado + '/' + i)

In [10]:
for grado in os.listdir('No Inpainting/No_Proc/Samsung'):
    for i in os.listdir('No Inpainting/No_Proc/Samsung/' + grado):
        img = cv2.imread('No Inpainting/No_Proc/Samsung/' + grado + '/' + i)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_t = circle_crop(img, 30)
        image = Image.fromarray(img_t, 'RGB')
        image.save('No Inpainting/Procesadas/Samsung/' + grado + '/' + i)